In [1]:
#This file's intention is to connect the stops for Madrid's Metro to our database. 
#It will be using Pandas to handle the dataframes (and to load into Oracle), os to check if the files exist, and oracledb to connect to the database itself.
#For the time being, we will only be focusing on stops.txt

In [2]:
import pandas as pd #for handling the dataframe
import os #used to check if file exsisted in the intended directory
import oracledb #to connect database
import MetroDeMadrid_db #importing db connection for security

In [3]:
file_path = 'data/MetroDeMadrid/gtfs/stops.txt' #directory

os.path.exists(file_path) #checking if the file path exists

True

In [4]:
stops = pd.read_csv('data/MetroDeMadrid/gtfs/stops.txt') #loading the file into a dataframe

stops.head() #printing to check

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,stop_timezone,wheelchair_boarding
0,par_4_1,1,PLAZA DE CASTILLA,Paseo de la Castellana 189,40.46690,-3.68917,A,http://www.crtm.es,0,est_90_21,NaN,0
1,acc_4_1_1,1,Plaza de Castilla,Paseo de la Castellana 189,40.46682,-3.68918,NaN,http://www.crtm.es,2,est_90_21,NaN,0
2,acc_4_1_1040,1,Ascensor,Plaza de Castilla 9,40.46555,-3.68877,NaN,http://www.crtm.es,2,est_90_21,NaN,0
3,acc_4_1_1043,1,Intercambiador Superficie,Paseo de la Castellana 191 B,40.46728,-3.68915,NaN,http://www.crtm.es,2,est_90_21,NaN,0
4,acc_4_1_1044,1,Ascensor,Paseo de la Castellana 189,40.46702,-3.68918,NaN,http://www.crtm.es,2,est_90_21,NaN,0


In [5]:
stops = stops.fillna("")  # Replace NaN with empty strings

In [6]:
stops.dtypes

stop_id                 object
stop_code                int64
stop_name               object
stop_desc               object
stop_lat               float64
stop_lon               float64
zone_id                 object
stop_url                object
location_type            int64
parent_station          object
stop_timezone           object
wheelchair_boarding      int64
dtype: object

In [7]:
#This function is about returning the maximum size of the values inside of each column.
def max_column_value_lengths(df):
    return stops.astype(str).apply(lambda col: col.str.len().max())

# Get the max size for each column
max_sizes = max_column_value_lengths(stops)

print(max_sizes)

stop_id                14
stop_code               3
stop_name              57
stop_desc              53
stop_lat                8
stop_lon                8
zone_id                 2
stop_url               18
location_type           1
parent_station          9
stop_timezone          13
wheelchair_boarding     1
dtype: int64


In [8]:
import importlib #Made changes to MetroDeMadrid_db, so it had to be reloaded.
import MetroDeMadrid_db #file that contains the db connection. Put it in a different directory for security. 

if importlib.reload(MetroDeMadrid_db):
    print("Module reloaded successfully!")

Module reloaded successfully!


In [9]:
#Reminder: the actual function that connects to the database is in a completely different directory from project folder for security.

#try: to test the connection to the database which will contain information about the madrid metro
try:
    connection = MetroDeMadrid_db.get_connection()
    print("Connection established successfully!")

    # Creating a cursor
    cursor = connection.cursor()

    #the string to feed into Oracle to create our table. We will be using stops.txt
    create_table_sql = """
    CREATE TABLE stops (
    stop_id VARCHAR2(16) PRIMARY KEY,
    stop_code NUMBER(4),
    stop_name VARCHAR2(64),
    stop_desc VARCHAR2(64),
    stop_lat NUMBER(9,6),
    stop_lon NUMBER(9,6),
    zone_id VARCHAR2(4),
    stop_url VARCHAR2(256),
    location_type NUMBER(1),
    parent_station VARCHAR2(16),
    stop_timezone VARCHAR2(16),
    wheelchair_boarding NUMBER(1)
)
"""

    #Passing the create table command into our Oracle database
    cursor.execute(create_table_sql)
    connection.commit()

    # Closing the cursor and connection
    cursor.close()
    connection.close()
    print("Connection closed successfully!")

#error handling function. Returns whatever the error was
except Exception as e:
    print(f"Error connecting to the database: {e}")



Connection established successfully!
Error connecting to the database: ORA-00955: name is already used by an existing object
Help: https://docs.oracle.com/error-help/db/ora-00955/


In [10]:
#Now that the table has been created, we are going to pass the information into the table STOPS
try:
    connection = MetroDeMadrid_db.get_connection()
    print("Connection established successfully!")

    # Creating a cursor
    cursor = connection.cursor()

    #SQL statement to insert our entire dataframe. Listed below is every single column
    sql_statement = """
    INSERT INTO STOPS (stop_id, stop_code, stop_name, stop_desc, stop_lat, stop_lon, zone_id, stop_url, location_type, parent_station, stop_timezone, wheelchair_boarding)
    VALUES (:1, :2, :3, :4, :5, :6, :7, :8, :9, :10, :11, :12)
    """
    #we are loading the entire dataframe into a tuple. Tuples are non-immutable, meaning nothing will change with the data in case anything happens. Standard practice.
    sql_data = [tuple(row) for row in stops.itertuples(index=False, name=None)]

    #executemany allows us to insert all the data at the same time
    cursor.executemany(sql_statement, sql_data)

    #commit executes the action itself
    connection.commit()

        # Closing the cursor and connection
    cursor.close()
    connection.close()
    print("Connection closed successfully!")

#error handling function. Returns whatever the error was
except Exception as e:
    print(f"Error connecting to the database: {e}")


Connection established successfully!
Error connecting to the database: ORA-00001: unique constraint (SYSTEM.SYS_C008223) violated
Help: https://docs.oracle.com/error-help/db/ora-00001/


In [11]:
print(stops.dtypes)

stop_id                 object
stop_code                int64
stop_name               object
stop_desc               object
stop_lat               float64
stop_lon               float64
zone_id                 object
stop_url                object
location_type            int64
parent_station          object
stop_timezone           object
wheelchair_boarding      int64
dtype: object
